# Crawl điểm chuẩn từ trang web tuyển sinh 247

In [2]:
import os
import time
import traceback
import pandas as pd
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import ElementClickInterceptedException, StaleElementReferenceException

In [3]:
from pathlib import Path

current_path = Path().resolve()
while current_path.name != "ADS_Final":
    if current_path.parent == current_path:
        raise FileNotFoundError("❌ Không tìm thấy thư mục gốc 'ADS_Final'")
    current_path = current_path.parent

os.chdir(current_path)

In [6]:
# os.chdir(r'D:\University\Applied Data Science\ADS_Final')

## Hàm bổ trợ

In [9]:
# ========== HÀM ĐÓNG QUẢNG CÁO ==========
def close_ads_if_present(driver, wait, max_wait=2):
    try:
        if "#google_vignette" in driver.current_url:
            driver.back()
            wait.until(lambda d: "#google_vignette" not in d.current_url)
            print("[ADS] Đã quay lại khi gặp quảng cáo Google Vignette.")
            time.sleep(1)
    except Exception as e:
        print("[ADS] Lỗi xử lý Google Vignette:", e)

    # Tắt các popup có thể xuất hiện
    close_xpath_candidates = [
        # modal phổ biến (Ant Design)
        "//div[contains(@class, 'ant-modal')]//button[contains(@class, 'ant-modal-close')]",
        # popup có nút Đóng viết hoa/thường/ẩn bên trong overlay
        "//button[translate(normalize-space(text()), 'ĐÓNG', 'đóng')='đóng']",
        "//button[contains(text(), '×')]",
        "//div[contains(@class, 'popup') or contains(@class, 'overlay')]//button",
    ]

    for xpath in close_xpath_candidates:
        try:
            popup_close = WebDriverWait(driver, max_wait).until(
                EC.presence_of_element_located((By.XPATH, xpath))
            )
            driver.execute_script("arguments[0].click();", popup_close)
            print(f"[ADS] ❌ Đã tự động tắt popup: {xpath}")
            time.sleep(1)
            break
        except Exception as e:
            continue

# ========== HÀM TẠO & RELOAD DRIVER ==========
def create_driver():
    options = uc.ChromeOptions()
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    profile_directory = "Profile"
    os.makedirs(profile_directory, exist_ok=True)
    options.user_data_dir = profile_directory
    return uc.Chrome(options=options)

def reload_driver():
    global driver, wait, action
    try:
        driver.quit()
    except:
        pass
    time.sleep(3)
    print("[RELOAD] Đang reload trình duyệt...")
    driver = create_driver()
    wait = WebDriverWait(driver, 15)
    action = ActionChains(driver)


## Crawl từ link https://diemthi.tuyensinh247.com/nganh-dao-tao.html#undefined

In [10]:
# ========== KHỞI TẠO ==========
reload_driver()
backup_file = "Data/nganh_dao_tao_backup_2.csv"
final_file = "Data/nganh_dao_tao_final_2.csv"
data = []

if os.path.exists(backup_file):
    df_backup = pd.read_csv(backup_file)
    data = df_backup.to_dict("records")
    print(f"[RESUME] Đã đọc {len(data)} dòng từ file backup.")

crawled_major_names = set(d["Ngành"] for d in data)

driver.get("https://diemthi.tuyensinh247.com/nganh-dao-tao.html#undefined")
close_ads_if_present(driver, wait)

try:
    see_more = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.XPATH, "//button[contains(text(),'Xem thêm')]"))
    )
    driver.execute_script("arguments[0].click();", see_more)
    print("[DEBUG] Đã click 'Xem thêm' để hiển thị thêm nhóm ngành.")
    time.sleep(2)
except:
    print("[WARN] Không thấy nút 'Xem thêm'.")

# ========== LẤY DANH SÁCH NHÓM NGÀNH ==========
group_data = []
printed_group_names = set()
seen_major_links = set()

try:
    print("[DEBUG] Đang chờ danh sách nhóm ngành...")
    wait.until(lambda d: len(d.find_elements(By.XPATH, '//div[@role="tablist"]/div')) >= 30)
    groups = driver.find_elements(By.XPATH, '//div[@role="tablist"]/div')
    for group_index in range(len(groups)):
        for attempt in range(3):
            try:
                groups = driver.find_elements(By.XPATH, '//div[@role="tablist"]/div')
                group = groups[group_index]
                group_name = group.find_element(By.TAG_NAME, "p").text.strip()
                driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", group)
                time.sleep(1.2)
                try:
                    action.move_to_element(group).click().perform()
                except:
                    driver.execute_script("arguments[0].click();", group)
                WebDriverWait(driver, 5).until(
                    lambda d: 'aria-expanded="true"' in group.get_attribute("outerHTML")
                )
                majors = group.find_element(
                    By.XPATH, './/div[contains(@class, "ant-collapse-content")]'
                ).find_elements(By.TAG_NAME, "a")
                if group_name not in printed_group_names:
                    print(f"[DEBUG] → Nhóm ngành: {group_name} có {len(majors)} ngành")
                    printed_group_names.add(group_name)
                for major in majors:
                    major_name = major.text.strip()
                    major_link = major.get_attribute("href")
                    if major_link not in seen_major_links:
                        seen_major_links.add(major_link)
                        group_data.append((group_name, major_name, major_link))
                break
            except:
                print(f"[WARN] Lỗi mở nhóm ngành {group_index}, thử lại ({attempt+1}/3)")
except Exception as e:
    print("[ERROR] ❌ Lỗi lấy danh sách ngành:", e)
    print(traceback.format_exc())

# ========== TRUY CẬP TỪNG NGÀNH VÀ LẤY DỮ LIỆU ==========
school_counter = len(data)
for idx, (group_name, major_name, major_link) in enumerate(group_data, 1):
    if major_name in crawled_major_names:
        continue
    print(f"[DEBUG] ({idx}/{len(group_data)}) Đang xử lý ngành: {major_name}")
    for try_load in range(2):
        try:
            driver.set_page_load_timeout(30)
            driver.get(major_link)
            close_ads_if_present(driver, wait)
            break
        except:
            print(f"[RETRY] Thử lại ngành {major_name}")
            if try_load == 1:
                continue
    try:
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, "ant-table-container")))
    except:
        print(f"[WARN] Không thấy bảng trường trong ngành {major_name}, bỏ qua.")
        continue

    # Mở rộng danh sách trường nếu có "Xem thêm"
    previous_count = 0
    while True:
        school_rows = driver.find_elements(By.XPATH, "//div[contains(@class, 'ant-table-container')]//tbody/tr")
        current_count = len(school_rows)
        if current_count > previous_count:
            previous_count = current_count
            try:
                see_more_button = WebDriverWait(driver, 3).until(
                    EC.element_to_be_clickable((By.XPATH, "//button[contains(@class, 'viewmore')]"))
                )
                driver.execute_script("arguments[0].click();", see_more_button)
                print(f"[DEBUG] Click 'Xem thêm' → {current_count}")
                time.sleep(2)
            except:
                print("[DEBUG] Không còn nút 'Xem thêm'.")
                break
        else:
            break

    school_info = []
    school_rows = driver.find_elements(By.XPATH, "//div[contains(@class, 'ant-table-container')]//tbody/tr")
    for row in school_rows:
        try:
            cols = row.find_elements(By.TAG_NAME, "td")
            if len(cols) >= 5:
                name = cols[1].text.strip()
                link = cols[4].find_element(By.TAG_NAME, "a").get_attribute("href")
                school_info.append((name, link))
        except:
            continue

    for i, (school_name, school_link) in enumerate(school_info, 1):
        print(f"[DEBUG]      ({i}) Trường: {school_name}")
        print(f"[DEBUG]        → Link: {school_link}")
        for retry in range(2):
            try:
                driver.set_page_load_timeout(30)
                driver.get(school_link)
                close_ads_if_present(driver, wait)
                wait.until(EC.presence_of_element_located((By.ID, "diem-thi-thpt")))
                table = driver.find_element(By.ID, "diem-thi-thpt")
                rows = table.find_elements(By.XPATH, ".//tbody/tr")
                print(f"[DEBUG]         Số dòng điểm tìm thấy: {len(rows)}")
                for row in rows:
                    cols = row.find_elements(By.TAG_NAME, "td")
                    if len(cols) >= 6:
                        data.append({
                            "Nhóm ngành": group_name,
                            "Ngành": major_name,
                            "Trường đào tạo": school_name,
                            "Mã ngành": cols[1].text.strip(),
                            "Tên ngành": cols[2].text.strip(),
                            "Tổ hợp môn": cols[3].text.strip(),
                            "Điểm chuẩn 2024": cols[4].text.strip(),
                            "Ghi chú": cols[5].text.strip()
                        })
                        school_counter += 1
                if school_counter % 10 == 0:
                    print(f"[INFO] Đã lấy được {school_counter} dòng dữ liệu...")
                break
            except:
                print("[RETRY] Thử lại trường...")
                reload_driver()
                break
    crawled_major_names.add(major_name)
    pd.DataFrame(data).to_csv(backup_file, index=False, encoding="utf-8-sig")

# ========== LƯU FILE CUỐI ==========
driver.quit()
df = pd.DataFrame(data)
if not df.empty:
    df.drop_duplicates(subset=["Mã ngành", "Trường đào tạo"], inplace=True)
    os.makedirs("Data", exist_ok=True)
    df.to_csv(final_file, index=False, encoding="utf-8-sig")
    print(f"✅ Đã crawl xong {school_counter} dòng dữ liệu. File đã lưu tại: {final_file}")
else:
    print("⚠️ Không có dữ liệu nào được lưu.")

[RELOAD] Đang reload trình duyệt...


C:\Users\WIN 10\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\undetected_chromedriver\__init__.py:339: UserWarning: using ChromeOptions.user_data_dir might stop working in future versions.use uc.Chrome(user_data_dir='/xyz/some/data') in case you need existing profile folder
  warnings.warn(


[WARN] Không thấy nút 'Xem thêm'.
[DEBUG] Đang chờ danh sách nhóm ngành...
[DEBUG] → Nhóm ngành: Kế toán - Kiểm toán có 2 ngành
[DEBUG] → Nhóm ngành: Tài chính - Ngân hàng - Bảo hiểm có 7 ngành
[DEBUG] → Nhóm ngành: Kinh tế - Quản trị kinh doanh - Thương Mại có 18 ngành
[DEBUG] → Nhóm ngành: Công nghệ thông tin - Tin học có 23 ngành
[DEBUG] → Nhóm ngành: Báo chí - Marketing - Quảng cáo - PR có 11 ngành
[DEBUG] → Nhóm ngành: Sư phạm - Giáo dục có 35 ngành
[DEBUG] → Nhóm ngành: Y - Dược có 20 ngành
[DEBUG] → Nhóm ngành: Bác sĩ thú y có 3 ngành
[DEBUG] → Nhóm ngành: Công an - Quân đội có 30 ngành
[DEBUG] → Nhóm ngành: Thiết kế đồ họa - Game - Đa phương tiện có 5 ngành
[DEBUG] → Nhóm ngành: Xây dựng - Kiến trúc - Giao thông có 22 ngành
[DEBUG] → Nhóm ngành: Ngoại giao - Ngoại ngữ có 24 ngành
[DEBUG] → Nhóm ngành: Ngoại thương - Xuất nhập khẩu - Kinh Tế quốc tế có 9 ngành
[DEBUG] → Nhóm ngành: Du lịch-Khách sạn có 12 ngành
[DEBUG] → Nhóm ngành: Ô tô - Cơ khí - Chế tạo có 14 ngành
[DEBUG] → 

## Crawl từ link https://diemthi.tuyensinh247.com/diem-chuan.html

In [11]:
# ========== CẤU HÌNH ==========
backup_file = "Data/diem_chuan_2024_backup.csv"
final_file = "Data/diem_chuan_2024_final.csv"
data = []

# ========== KHỞI TẠO ==========
reload_driver()
driver.get("https://diemthi.tuyensinh247.com/diem-chuan.html")
close_ads_if_present(driver, wait)

# ========== SCROLL TẢI TOÀN BỘ TRƯỜNG ==========
try:
    scroll_box = driver.find_element(By.CLASS_NAME, "list-schol-box")
    previous_count = 0
    while True:
        school_tags = driver.find_elements(By.CSS_SELECTOR, ".list-schol-box a")
        current_count = len(school_tags)
        if current_count > previous_count:
            previous_count = current_count
            driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scroll_box)
            time.sleep(1)
        else:
            break
except Exception as e:
    print("[WARN] Không scroll được danh sách trường:", e)

# ========== LẤY DANH SÁCH LINK TRƯỜNG ==========
school_links = []
school_tags = driver.find_elements(By.CSS_SELECTOR, ".list-schol-box a")
for tag in school_tags:
    href = tag.get_attribute("href")
    name = tag.text.strip()
    if href:
        school_links.append((name, href))
print(f"[INFO] Tìm thấy {len(school_links)} trường.")

# ========== KHỞI TẠO DỮ LIỆU BACKUP ==========
crawled_school_names = set()
if os.path.exists(backup_file):
    df_backup = pd.read_csv(backup_file)
    data = df_backup.to_dict("records")
    crawled_school_names = set(row["Trường đào tạo"] for row in data)
    print(f"[RESUME] Đã load {len(data)} dòng từ backup. {len(crawled_school_names)} trường đã crawl.")

# ========== CRAWL DỮ LIỆU THEO TRƯỜNG ==========
for idx, (school_name, link) in enumerate(school_links, 1):
    if school_name in crawled_school_names:
        print(f"[SKIP] {school_name} đã crawl, bỏ qua.")
        continue

    print(f"[{idx}/{len(school_links)}] 📚 Trường: {school_name} - {link}")
    try:
        driver.set_page_load_timeout(30)
        driver.get(link)
        close_ads_if_present(driver, wait)

        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "diem-thi-thpt"))
        )
        table = driver.find_element(By.ID, "diem-thi-thpt")
        rows = table.find_elements(By.XPATH, ".//tbody/tr")
        print(f"    → Tìm thấy {len(rows)} dòng.")

        for row in rows:
            cols = row.find_elements(By.TAG_NAME, "td")
            if len(cols) >= 5:
                data.append({
                    "Trường đào tạo": school_name,
                    "Mã ngành": cols[1].text.strip(),
                    "Tên ngành": cols[2].text.strip(),
                    "Tổ hợp môn": cols[3].text.strip(),
                    "Điểm chuẩn": cols[4].text.strip(),
                    "Ghi chú": cols[5].text.strip() if len(cols) > 5 else ""
                })

        # 🔄 Lưu backup sau mỗi trường
        pd.DataFrame(data).to_csv(backup_file, index=False, encoding="utf-8-sig")
        crawled_school_names.add(school_name)
        print(f"[SAVE] Đã backup sau trường: {school_name}")

    except Exception as e:
        print(f"[WARN] Không thể crawl trường {school_name}: {e}")
        traceback.print_exc()

# ========== LƯU FILE FINAL ==========
df = pd.DataFrame(data)
os.makedirs("Data", exist_ok=True)
df.drop_duplicates(subset=["Trường đào tạo", "Mã ngành", "Tổ hợp môn"], inplace=True)
df.to_csv(final_file, index=False, encoding="utf-8-sig")
print(f"✅ Đã crawl xong {len(df)} dòng dữ liệu. File đã lưu tại: {final_file}")

[RELOAD] Đang reload trình duyệt...
[INFO] Tìm thấy 300 trường.
[1/300] 📚 Trường: KHA - Đại Học Kinh Tế Quốc Dân - https://diemthi.tuyensinh247.com/diem-chuan/dai-hoc-kinh-te-quoc-dan-KHA.html
    → Tìm thấy 66 dòng.
[SAVE] Đã backup sau trường: KHA - Đại Học Kinh Tế Quốc Dân
[2/300] 📚 Trường: BKA - Đại Học Bách Khoa Hà Nội - https://diemthi.tuyensinh247.com/diem-chuan/dai-hoc-bach-khoa-ha-noi-BKA.html
    → Tìm thấy 64 dòng.
[SAVE] Đã backup sau trường: BKA - Đại Học Bách Khoa Hà Nội
[3/300] 📚 Trường: HTC - Học Viện Tài chính - https://diemthi.tuyensinh247.com/diem-chuan/hoc-vien-tai-chinh-HTC.html
    → Tìm thấy 14 dòng.
[SAVE] Đã backup sau trường: HTC - Học Viện Tài chính
[4/300] 📚 Trường: NHH - Học Viện Ngân Hàng - https://diemthi.tuyensinh247.com/diem-chuan/hoc-vien-ngan-hang-NHH.html
    → Tìm thấy 30 dòng.
[SAVE] Đã backup sau trường: NHH - Học Viện Ngân Hàng
[5/300] 📚 Trường: NTH - Trường Đại học Ngoại thương - https://diemthi.tuyensinh247.com/diem-chuan/dai-hoc-ngoai-thuong-c

Traceback (most recent call last):
  File "C:\Users\WIN 10\AppData\Local\Temp\ipykernel_3792\3382215369.py", line 77, in <module>
    pd.DataFrame(data).to_csv(backup_file, index=False, encoding="utf-8-sig")
  File "C:\Users\WIN 10\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\pandas\util\_decorators.py", line 333, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\WIN 10\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\pandas\core\generic.py", line 3967, in to_csv
    return DataFrameRenderer(formatter).to_csv(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\WIN 10\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\pandas\io\formats\format.py", line 1014, in to_csv
    csv_formatter.save()
  File "C:\Use

    → Tìm thấy 21 dòng.
[WARN] Không thể crawl trường YHB - Trường Đại Học Y Hà Nội: [Errno 13] Permission denied: 'Data/diem_chuan_2024_backup.csv'
[8/300] 📚 Trường: BVH - Học Viện Công Nghệ Bưu Chính Viễn Thông - https://diemthi.tuyensinh247.com/diem-chuan/hoc-vien-cong-nghe-buu-chinh-vien-thong-phia-bac-BVH.html


Traceback (most recent call last):
  File "C:\Users\WIN 10\AppData\Local\Temp\ipykernel_3792\3382215369.py", line 77, in <module>
    pd.DataFrame(data).to_csv(backup_file, index=False, encoding="utf-8-sig")
  File "C:\Users\WIN 10\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\pandas\util\_decorators.py", line 333, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\WIN 10\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\pandas\core\generic.py", line 3967, in to_csv
    return DataFrameRenderer(formatter).to_csv(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\WIN 10\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\pandas\io\formats\format.py", line 1014, in to_csv
    csv_formatter.save()
  File "C:\Use

    → Tìm thấy 22 dòng.
[WARN] Không thể crawl trường BVH - Học Viện Công Nghệ Bưu Chính Viễn Thông: [Errno 13] Permission denied: 'Data/diem_chuan_2024_backup.csv'
[9/300] 📚 Trường: SPH - Trường Đại Học Sư Phạm Hà Nội - https://diemthi.tuyensinh247.com/diem-chuan/dai-hoc-su-pham-ha-noi-SPH.html


...

    → Tìm thấy 44 dòng.
[SAVE] Đã backup sau trường: DNV - Trường Đại học Nội vụ Hà Nội
[295/300] 📚 Trường: VPH - Trường Sĩ Quan Kĩ Thuật Quân Sự - Hệ Quân sự - Đại Học Trần Đại Nghĩa - https://diemthi.tuyensinh247.com/diem-chuan/truong-si-quan-ki-thuat-quan-su-he-quan-su-dai-hoc-tran-dai-nghia-VPH.html
    → Tìm thấy 2 dòng.
[SAVE] Đã backup sau trường: VPH - Trường Sĩ Quan Kĩ Thuật Quân Sự - Hệ Quân sự - Đại Học Trần Đại Nghĩa
[296/300] 📚 Trường: DSD - Trường Đại Học Sân Khấu Điện Ảnh TPHCM - https://diemthi.tuyensinh247.com/diem-chuan/dai-hoc-san-khau-dien-anh-tphcm-DSD.html
[WARN] Không thể crawl trường DSD - Trường Đại Học Sân Khấu Điện Ảnh TPHCM: Message: 
Stacktrace:
	GetHandleVerifier [0x0xf73763+63299]
	GetHandleVerifier [0x0xf737a4+63364]
	(No symbol) [0x0xda1113]
	(No symbol) [0x0xde987e]
	(No symbol) [0x0xde9c1b]
	(No symbol) [0x0xe32212]
	(No symbol) [0x0xe0e5c4]
	(No symbol) [0x0xe2fa4a]
	(No symbol) [0x0xe0e376]
	(No symbol) [0x0xddd6e0]
	(No symbol) [0x0xdde544]
	GetHan

Traceback (most recent call last):
  File "C:\Users\WIN 10\AppData\Local\Temp\ipykernel_3792\3382215369.py", line 57, in <module>
    WebDriverWait(driver, 10).until(
  File "C:\Users\WIN 10\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\selenium\webdriver\support\wait.py", line 146, in until
    raise TimeoutException(message, screen, stacktrace)
selenium.common.exceptions.TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x0xf73763+63299]
	GetHandleVerifier [0x0xf737a4+63364]
	(No symbol) [0x0xda1113]
	(No symbol) [0x0xde987e]
	(No symbol) [0x0xde9c1b]
	(No symbol) [0x0xe32212]
	(No symbol) [0x0xe0e5c4]
	(No symbol) [0x0xe2fa4a]
	(No symbol) [0x0xe0e376]
	(No symbol) [0x0xddd6e0]
	(No symbol) [0x0xdde544]
	GetHandleVerifier [0x0x11ce033+2531347]
	GetHandleVerifier [0x0x11c9332+2511634]
	GetHandleVerifier [0x0xf99eda+220858]
	GetHandleVerifier [0x0xf8a528+156936]
	GetHandleVerifier [0x0xf90c5d+18335

    → Tìm thấy 23 dòng.
[SAVE] Đã backup sau trường: DDA - Trường Đại Học Công Nghệ Đông Á
[298/300] 📚 Trường: BUV - Trường Đại Học Anh Quốc Việt Nam - https://diemthi.tuyensinh247.com/diem-chuan/dai-hoc-anh-quoc-viet-nam-BUV.html
[WARN] Không thể crawl trường BUV - Trường Đại Học Anh Quốc Việt Nam: Message: 
Stacktrace:
	GetHandleVerifier [0x0xf73763+63299]
	GetHandleVerifier [0x0xf737a4+63364]
	(No symbol) [0x0xda1113]
	(No symbol) [0x0xde987e]
	(No symbol) [0x0xde9c1b]
	(No symbol) [0x0xe32212]
	(No symbol) [0x0xe0e5c4]
	(No symbol) [0x0xe2fa4a]
	(No symbol) [0x0xe0e376]
	(No symbol) [0x0xddd6e0]
	(No symbol) [0x0xdde544]
	GetHandleVerifier [0x0x11ce033+2531347]
	GetHandleVerifier [0x0x11c9332+2511634]
	GetHandleVerifier [0x0xf99eda+220858]
	GetHandleVerifier [0x0xf8a528+156936]
	GetHandleVerifier [0x0xf90c5d+183357]
	GetHandleVerifier [0x0xf7b6c8+95912]
	GetHandleVerifier [0x0xf7b870+96336]
	GetHandleVerifier [0x0xf6664a+9770]
	BaseThreadInitThunk [0x0x7574fcc9+25]
	RtlGetAppContai

Traceback (most recent call last):
  File "C:\Users\WIN 10\AppData\Local\Temp\ipykernel_3792\3382215369.py", line 57, in <module>
    WebDriverWait(driver, 10).until(
  File "C:\Users\WIN 10\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\selenium\webdriver\support\wait.py", line 146, in until
    raise TimeoutException(message, screen, stacktrace)
selenium.common.exceptions.TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x0xf73763+63299]
	GetHandleVerifier [0x0xf737a4+63364]
	(No symbol) [0x0xda1113]
	(No symbol) [0x0xde987e]
	(No symbol) [0x0xde9c1b]
	(No symbol) [0x0xe32212]
	(No symbol) [0x0xe0e5c4]
	(No symbol) [0x0xe2fa4a]
	(No symbol) [0x0xe0e376]
	(No symbol) [0x0xddd6e0]
	(No symbol) [0x0xdde544]
	GetHandleVerifier [0x0x11ce033+2531347]
	GetHandleVerifier [0x0x11c9332+2511634]
	GetHandleVerifier [0x0xf99eda+220858]
	GetHandleVerifier [0x0xf8a528+156936]
	GetHandleVerifier [0x0xf90c5d+18335

    → Tìm thấy 4 dòng.
[SAVE] Đã backup sau trường: BMU - Trường Đại học Y Dược Buôn Ma Thuột
[300/300] 📚 Trường: DDG - Khoa Giáo dục thể chất Đà Nẵng - https://diemthi.tuyensinh247.com/diem-chuan/khoa-giao-duc-the-chat-da-nang-DDG.html
    → Tìm thấy 1 dòng.
[SAVE] Đã backup sau trường: DDG - Khoa Giáo dục thể chất Đà Nẵng
✅ Đã crawl xong 5925 dòng dữ liệu. File đã lưu tại: Data/diem_chuan_2024_final.csv


## Tại sao phải crawl từ 2 link khác nhau?

- Nhóm nhận thấy sau khi chỉ crawl từ link https://diemthi.tuyensinh247.com/nganh-dao-tao.html#undefined thì thiếu khá nhiều tổ hợp môn và điểm chuẩn.
- Nên trong khi tìm hiểu thêm thì nhóm đã phát hiện ra https://diemthi.tuyensinh247.com/diem-chuan.html có chứa đầy đủ điểm chuẩn và tổ hợp môn hơn nhưng bù lại bị thiếu nhóm ngành và ngành.
- Vì vậy nhóm quyết định crawl cả 2 nguồn sau đó merge lại rồi tiền xử lý để có được đầy đủ data.